In [ ]:
!pip install tensorflow
!pip install tf-slim
!pip install pycocotools
!pip install git+https://github.com/tensorflow/models.git#egg=object_detection&subdirectory=research/object_detection/packages/tf2


  Cloning https://github.com/tensorflow/models.git to /tmp/pip-install-0062m6f7/object-detection_c6dcd20cce8747718e7467908a8b4b15
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/models.git /tmp/pip-install-0062m6f7/object-detection_c6dcd20cce8747718e7467908a8b4b15
  Resolved https://github.com/tensorflow/models.git to commit aa90cd5c0d7e1f686891f805e4b564d0bc11dfbf
ERROR: object_detection from git+https://github.com/tensorflow/models.git#egg=object_detection does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
# 1. Clone TensorFlow Models repository
!git clone --depth 1 https://github.com/tensorflow/models

# 2. Install dependencies
!pip install tf-slim pycocotools

# 3. Compile protobufs
!apt-get install -y protobuf-compiler
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

# 4. Install the Object Detection API
!cp models/research/object_detection/packages/tf2/setup.py models/research/
!pip install models/research/

# 5. Set PYTHONPATH
import os
os.environ['PYTHONPATH'] += ':/content/models:/content/models/research:/content/models/research/slim'


fatal: destination path 'models' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Processing ./models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697325 sha256=b9cf5af9beef1844f6756dcffc266748ad0847e249291784098463bb6e0bdcd1
  Stored in directory: /tmp/pip-ephem-wheel-cache-aorvoblr/wheels/95/4a/63/b2d36ca06eab841de19a38993ffaf2beac152a44539bc642a6
Successfully built object_detection
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Uninstalling object_detection-0.1:
      Successfully uninstalled object_detection-0.1


In [ ]:
!pip install tabulate

In [ ]:
# 1. Clone TensorFlow Models
!git clone --depth 1 https://github.com/tensorflow/models

# 2. Install dependencies
!pip install tf-slim pycocotools
!pip install protobuf==3.20.*

# 3. Compile protobuf
!apt-get install -y protobuf-compiler
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

# 4. Install the Object Detection API
!cp models/research/object_detection/packages/tf2/setup.py models/research/
!pip install models/research/

# 5. Set PYTHONPATH
import os
os.environ['PYTHONPATH'] += ':/content/models:/content/models/research:/content/models/research/slim'


fatal: destination path 'models' already exists and is not an empty directory.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires grpcio>=1.71.0, but you have grpcio 1.65.5 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0,

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Processing ./models/research
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 8.1 MB/s eta 0:00:00
  Created wheel for object_detection: filename=object_detection-0.1-py3-none-any.whl size=1697325 sha256=3e5c354f7c641eb87b06b673f42701b2cc2968904c2684df731c9635f211af6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-hy2_0emy/wheels/95/4a/63/b2d36ca06eab841de19a38993ffaf2beac152a44539bc642a6
Successfully built object_detection
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: object_detection
    Found existing installation: object_detection 0.1
    Can't uninstall 'obj

In [ ]:
# Downgrade protobuf ke versi yang kompatibel (3.20.x)
!pip install protobuf==3.20.*

# Restart runtime diperlukan agar perubahan berlaku
import os
os.kill(os.getpid(), 9)


In [ ]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from collections import defaultdict
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from tabulate import tabulate


In [ ]:

# Konstanta
PIXEL_AREA_TO_M2 = 0.0007095  # m²/pixel²
CARBON_FRACTION = 0.47
CO2_CONVERSION = 3.67

# Parameter allometrik
ALLOMETRIC_PARAMS = {
    "Rhizophora": {"a": 0.251, "b": 1.21},
    "Avicennia": {"a": 0.184, "b": 1.29},
    "Nypa": {"a": 0.126, "b": 1.15}
}

def calculate_area_from_box(w, h):
    pixel_area = w * h
    return pixel_area * PIXEL_AREA_TO_M2, pixel_area

def estimate_carbon(area_m2, genus):
    if genus not in ALLOMETRIC_PARAMS:
        return 0, 0, 0
    a = ALLOMETRIC_PARAMS[genus]["a"]
    b = ALLOMETRIC_PARAMS[genus]["b"]
    B = a * (area_m2 ** b)
    C = B * CARBON_FRACTION
    C_abs = C * CO2_CONVERSION
    return B, C, C_abs



In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from collections import defaultdict
from object_detection.utils import label_map_util, visualization_utils as viz_utils
from tabulate import tabulate
import matplotlib.pyplot as plt

# Fungsi estimasi karbon
def calculate_area_from_box(width_px, height_px, pixel_size_cm=5):
    area_px = width_px * height_px
    pixel_area_m2 = (pixel_size_cm / 100) ** 2
    area_m2 = area_px * pixel_area_m2
    return area_m2, area_px

def estimate_carbon(area_m2, genus):
    if genus == "Rhizophora":
        B = 0.251 * (area_m2 ** 1.21)
    elif genus == "Avicennia":
        B = 0.184 * (area_m2 ** 1.29)
    elif genus == "Nypa":
        B = 0.126 * (area_m2 ** 1.15)
    else:
        B = 0
    C = 0.464 * B
    CO2 = C * 3.67
    return B, C, CO2

# Path model dan file
MODEL_PATH = "/content/drive/MyDrive/MANGROVE FIX/saved_model"
LABEL_MAP_PATH = "/content/drive/MyDrive/MANGROVE FIX/label_map_center.pbtxt"
IMAGE_PATH = "/content/nypaa.jpg"

# Load model dan label map
detect_fn = tf.saved_model.load(MODEL_PATH)
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

# Load gambar
image = Image.open(IMAGE_PATH)
image_np = np.array(image)
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

# Inference
detections = detect_fn.signatures["serving_default"](input_tensor)
num_detections = int(detections.pop("num_detections"))
detections = {k: v[0, :num_detections].numpy() for k, v in detections.items()}
detections["num_detections"] = num_detections
detections["detection_classes"] = detections["detection_classes"].astype(np.int64)

# Persiapan data
image_h, image_w, _ = image_np.shape
class_data = defaultdict(lambda: {
    "count": 0,
    "areas": [],
    "biomass": 0,
    "carbon": 0,
    "co2": 0,
    "confidences": [],
    "details": []
})

# Proses deteksi valid
valid_idx = 0
for idx in range(num_detections):
    score = detections["detection_scores"][idx]
    if score < 0.3:
        continue

    class_id = detections["detection_classes"][idx]
    class_label = category_index[class_id]["name"] if class_id in category_index else "Unknown"
    y_min, x_min, y_max, x_max = detections["detection_boxes"][idx]
    x_min *= image_w
    x_max *= image_w
    y_min *= image_h
    y_max *= image_h

    w = x_max - x_min
    h = y_max - y_min
    area_m2, area_px = calculate_area_from_box(w, h)

    class_label_lower = class_label.lower()
    if "rhizophora" in class_label_lower:
        genus = "Rhizophora"
    elif "avicennia" in class_label_lower:
        genus = "Avicennia"
    elif "nypa" in class_label_lower:
        genus = "Nypa"
    else:
        genus = "Unknown"

    B, C, CO2 = estimate_carbon(area_m2, genus)

    data = class_data[class_label]
    valid_idx += 1
    data["count"] += 1
    data["areas"].append(area_m2)
    data["biomass"] += B
    data["carbon"] += C
    data["co2"] += CO2
    data["confidences"].append(score)
    data["details"].append({
        "box_index": valid_idx,
        "area_m2": area_m2,
        "area_px": area_px,
        "biomass": B,
        "carbon": C,
        "co2": CO2,
        "confidence": score
    })

# Cetak hasil
print(f"\n{'='*30} HASIL DETEKSI {'='*30}")
for class_label, data in class_data.items():
    print(f"\nKelas: {class_label}")
    print(f"Jumlah objek : {data['count']}")
    print(f"Total biomassa         : {data['biomass']:.4f} kg")
    print(f"Total kandungan karbon : {data['carbon']:.4f} kg")
    print(f"Total CO2              : {data['co2']:.4f} kg")
    print(f"Rata-rata confidence   : {np.mean(data['confidences']):.2f}\n")

    table_data = []
    for detail in data["details"]:
        table_data.append([
            detail["box_index"],
            f"{detail['area_m2']:.4f}",
            f"{int(detail['area_px'])}",
            f"{detail['biomass']:.4f}",
            f"{detail['carbon']:.4f}",
            f"{detail['co2']:.4f}",
            f"{detail['confidence']:.2f}"
        ])
    headers = ["Objek", "Luas (m²)", "Luas (px²)", "Biomassa (kg)", "Kandungan Karbon (kg)", "CO2 (kg)", "Confidence"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Visualisasi hasil deteksi
image_with_detections = image_np.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
    image_with_detections,
    detections["detection_boxes"],
    detections["detection_classes"],
    detections["detection_scores"],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.3
)

# Tambahkan label khusus
image_pil = Image.fromarray(image_with_detections)
draw = ImageDraw.Draw(image_pil)
font = ImageFont.load_default()
obj_counter = 1

for i in range(num_detections):
    score = detections["detection_scores"][i]
    if score < 0.3:
        continue

    class_id = detections["detection_classes"][i]
    class_label = category_index[class_id]["name"]
    y_min, x_min, y_max, x_max = detections["detection_boxes"][i]
    left, right, top, bottom = x_min * image_w, x_max * image_w, y_min * image_h, y_max * image_h

    width_px = right - left
    height_px = bottom - top
    area_m2, _ = calculate_area_from_box(width_px, height_px)

    if "rhizophora" in class_label.lower():
        genus = "Rhizophora"
    elif "avicennia" in class_label.lower():
        genus = "Avicennia"
    elif "nypa" in class_label.lower():
        genus = "Nypa"
    else:
        genus = "Unknown"

    _, _, CO2 = estimate_carbon(area_m2, genus)

    label = f"Objek {obj_counter} | {genus} | CO2: {CO2:.2f} kg"
    obj_counter += 1

    text_bbox = draw.textbbox((left, top), label, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

        # Gambar latar putih dan teks hitam di atas bounding box
    draw.rectangle([(left, top - text_height), (left + text_width, top)], fill="white")
    draw.text((left, top - text_height), label, fill="black", font=font)

# Tampilkan hasil visualisasi
print("\nHasil visualisasi deteksi:")
plt.figure(figsize=(12, 8))
plt.imshow(image_pil)
plt.axis("off")
plt.tight_layout()
plt.show()

# Simpan ke file
output_path = "/content/mangrove_detected.jpg"
image_pil.save(output_path)
print(f"Hasil deteksi disimpan ke: {output_path}")



============================== HASIL DETEKSI ==============================

Kelas: nypa
Jumlah objek : 9
Total biomassa         : 47.2913 kg
Total kandungan karbon : 21.9432 kg
Total CO2              : 80.5315 kg
Rata-rata confidence   : 0.67

+---------+-------------+--------------+-----------------+-------------------------+------------+--------------+
|   Objek |   Luas (m²) |   Luas (px²) |   Biomassa (kg) |   Kandungan Karbon (kg) |   CO2 (kg) |   Confidence |
+=========+=============+==============+=================+=========================+============+==============+
|       1 |     27.3619 |        10944 |          5.6636 |                  2.6279 |     9.6444 |         0.91 |
+---------+-------------+--------------+-----------------+-------------------------+------------+--------------+
|       2 |     19.1608 |         7664 |          3.7596 |                  1.7445 |     6.4022 |         0.87 |
+---------+-------------+--------------+-----------------+------------------